In [ ]:
import sys
print(sys.executable) 

In [ ]:
# ! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install pandas
# ! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install icecream
# ! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install nltk
# ! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install plotly
# ! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install pytorch_pretrained_bert pytorch-nlp
# ! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install keras
# ! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install tensorflow
# ! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install transformers
! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install ipynb

In [ ]:
import pandas as pd
import numpy as np
import random as rn
from tqdm.notebook import tqdm
from icecream import ic 

import nltk
# from nltk.tokenize import regexp_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

# imports from shudima
# import torch
# from pytorch_pretrained_bert import BertModel
# from torch import nn
# from torchnlp.datasets import imdb_dataset
# from pytorch_pretrained_bert import BertTokenizer

# imports from colbert
# import tensorflow as tf
# import bert_tokenization as tokenization
# import tensorflow.keras.backend as K
# from tensorflow import keras
# from transformers import *
from transformers import BertTokenizer

# imports from shudima
# from keras.preprocessing.sequence import pad_sequences
# from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# from torch.optim import Adam
# from torch.nn.utils import clip_grad_norm_
# from IPython.display import clear_output

# import re
# import os

In [ ]:
#init 
train_count = 1000 #8000
# test_count = 

MAX_SENTENCE_LENGTH = 128 #colbert=20
MAX_SENTENCES = 5
MAX_LENGTH = 100

In [ ]:
# preprocessing for BERT

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenizer.tokenize('Hi my nambe is Dima')
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], train['text']))

In [ ]:
from ipynb.fs.full.utils import *
from ipynb.fs.full.datasets import *
from ipynb.fs.full.architectures import *
from ipynb.fs.full.train_test import *

In [ ]:
sys.path.append(".")

# if seq length is 128 batch size should be 32
traindata = torch.utils.data.DataLoader(dataset= Hahackathon("data/train.csv", basenet= 'bert'), batch_size= 32, shuffle= True)
# ic("Size of the dataset",len(traindata.dataset))
model = multitask_lstm_fc('bert')

In [ ]:
# parser.add_argument('--lr_max',            type = float,          default = 0.00002,      help = 'learning rate (default: 0.01)')
learning_rate=0.01
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)


In [ ]:

from sklearn.metrics import roc_auc_score, mean_squared_error, f1_score, accuracy_score
 
# mse_criterion = torch.nn.MSELoss()
ce_criterion  = torch.nn.CrossEntropyLoss()
ic(ce_criterion)

def concat_output1(output, device):
    temp = torch.tensor([]).to(device)

    if output.shape[1] == 2:
        ic()
        temp = torch.argmax(output[:, 0:2], dim=1).view(-1,1)
        ic(temp)
        return temp

    temp = torch.argmax(output[:, 0:2], dim=1).view(-1,1)
    temp = torch.cat((temp, torch.argmax(output[:, 2:4], dim= 1).view(-1,1)), dim= 1)
    temp = torch.cat((temp, output[:, 4].view(-1,1)), dim= 1)
    temp = torch.cat((temp, output[:, 5].view(-1,1)), dim= 1)
    return temp

def loss_func1(output, target, device):

    humor_id = (target[:, 0]==1)
    target   = target.float()
    ic(output[:, 0:2])
    
    loss1 = ce_criterion(output[:, 0:2], target[:, 0].long())
    loss=loss1
    
    # this block of generating temp is verified throughly
    temp = concat_output1(output, device)
    ic('outside concat_output ', temp)
    temp = temp.detach()

    return loss, temp

def train1(model, dataloader, optimizer, device):
#     ic('1')
    model.train()
#     ic('2')
    train_loss = AverageMeter()

    for batch_id, batch_data in enumerate(dataloader):
        data       = batch_data[0]
        target     = batch_data[1].to(device)
#         ic(data)
#         ic(target)

        input_id   = data[0].to(device).squeeze()
        mask_id    = data[1].to(device).squeeze()
        segment_id = data[2].to(device).squeeze()
#         ic(input_id)
#         ic(mask_id)
#         ic(segment_id)
        
        output = model(input_id, mask_id, segment_id)
#         ic('output model')
        
        loss, _ = loss_func1(output, target, device= device)
        ic('outside loss_func')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss.update(loss.item(), target.shape[0])

    return train_loss.avg

In [1]:
use_cuda= False
device   = torch.device("cuda" if use_cuda else "cpu")
model = model.to(device)

# ic(traindata)
# ic(optimizer)

train_loss = train1(model, traindata, optimizer, device)
ic(train_loss)

NameError: name 'torch' is not defined